In [4]:
import numpy as np 
import scipy

In [6]:
#41連立方程式の表現
#43連立方程式を解く
A1=np.array([
    [1,5],
    [5,6]
])
B1=np.array([
    [3],
    [4]
])
print(np.linalg.solve(A1,B1))

[[0.10526316]
 [0.57894737]]


In [7]:
#42連率方程式の表現２
#43連立方程式を解く
A2=np.array([
    [1,5,3],
    [5,4,2],
    [4,3,5]
])
B2=np.array([
    [3],
    [4],
    [7]
])

print(np.linalg.solve(A2,B2))

[[ 0.48648649]
 [-0.16216216]
 [ 1.10810811]]


In [26]:
#行列のランク２
def row_swap(A,i,j):
    A[i],A[j]=A[j],A[i]

def row_scale(A,i,k):
    if k==0:
        raise ValueError('k must be non-zero')
    A[i]=[k*x for x in A[i]]

def row_add(A,src,dest,k):
    A[dest]=[d+k*s for d,s in zip(A[dest],A[src])]

def gaussian_elmination(mat):
    m,n=len(mat),len(mat[0])
    pivot_cols=[]
    row=0
    for col in range(n):
        sel=max(range(row,m),key=lambda r: abs(mat[r][col]))
        if abs(mat[sel][col])<1**(-12):
            continue
        row_swap(mat,row,sel)
        pivot_cols.append(col)
        
        pivot=mat[row][col]
        mat[row]=[x/pivot for x in mat[row]]
        
        for r in range(m):
            if r==row:
                continue
            factor=mat[r][col]
            row_add(mat,row,r,-factor)
        row+=1
        if row==m:
            break
    return pivot_cols

## 1. 目的と概要

連立一次方程式

$$
\begin{cases}
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n = b_1 \\
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n = b_2 \\
\quad \vdots \\
a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mn}x_n = b_m
\end{cases}
$$

を行列を使って書き表すと、

$$
A\mathbf{x} = \mathbf{b}, \quad
A = \begin{pmatrix}
a_{11} & a_{12} & \dots & a_{1n}\\
a_{21} & a_{22} & \dots & a_{2n}\\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \dots & a_{mn}
\end{pmatrix},\quad
\mathbf{x} = \begin{pmatrix}
x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix},\quad
\mathbf{b} = \begin{pmatrix}
b_1 \\ b_2 \\ \vdots \\ b_m
\end{pmatrix}.
$$

ガウスの消去法では、この拡大係数行列

$$
(A|\mathbf{b}) = 
\begin{pmatrix}
a_{11} & a_{12} & \dots & a_{1n} & b_1\\
a_{21} & a_{22} & \dots & a_{2n} & b_2\\
\vdots & \vdots & \ddots & \vdots & \vdots\\
a_{m1} & a_{m2} & \dots & a_{mn} & b_m
\end{pmatrix}
$$

に対して「行の入れ替え」「行の定数倍」「行どうしの加減」といった**基本変形**を行い、**段階的に 0 を作っていく** ことで、下図のような「上三角形」「階段状」の形に変形し、そこから順次解を求める(後退代入)という流れになります。

---

## 2. 基本操作

ガウスの消去法においては、以下の **3種類の行操作** (行基本変換) が正当化されており、これらを自由に組み合わせられます。

1. **行の入れ替え (Pivoting)**

   * 行 $i$ と行 $j$ を入れ替える。
   * 例: 「上の方に 0 でないピボット(基準となる係数)を持ってくる」ために行う。
2. **行の定数倍**

   * 行 $i$ を定数倍($\neq 0$)する。
3. **行どうしの加減**

   * 行 $j$ に「行 $i$ の定数倍」を加える。
   * 例: $R_j \leftarrow R_j - \alpha R_i$ のようにして、行の一部要素を 0 にする。

拡大行列でこれらの操作を行っても、方程式系の解は変わりません(行入れ替えの場合は順番が変わるだけで、同じ解集合を表す)。
**最終目標** は、「対角要素以外を消して」または「段階的に不要な係数を消して」**階段形 (row echelon form)** にすることです。

---

## 3. アルゴリズムの手順

簡単な例として、$2\times 2$ の連立方程式をガウス消去する流れを見てみます。

$$
\begin{cases}
a_{11} x_1 + a_{12} x_2 = b_1 \\
a_{21} x_1 + a_{22} x_2 = b_2
\end{cases}
\quad\Longleftrightarrow\quad
\begin{pmatrix}
a_{11} & a_{12} & b_1\\
a_{21} & a_{22} & b_2
\end{pmatrix}.
$$

### 例: 2×2の場合

1. **行の入れ替え (必要なら)**

   * もし $a_{11}=0$ なら、下の行と入れ替えて $a_{11} \neq 0$ とする。
2. **1列目の下の要素を消す**

   * ピボットを $a_{11}$ として、

     $$
       R_2 \leftarrow R_2 - \frac{a_{21}}{a_{11}}R_1
     $$
   * すると行列は

     $$
       \begin{pmatrix}
         a_{11} & a_{12} & b_1\\
         0      & a_{22}'& b_2'
       \end{pmatrix}
     $$

     となり、1列目の下要素が0になる。
3. **後退代入**

   * 2行目から $ x_2 = b_2'/a_{22}'$ を求め、1行目から $x_1 = (b_1 - a_{12} x_2)/a_{11}$ を求める。

以上が 2×2 のガウス消去法の基本です。
**大規模な (n×n や m×n) の場合でも、ピボットを決めてそれ以下・右の要素を消していく、というステップを繰り返すだけ** です。

---

## 4. 一般の $n \times n$ 行列 (または $m \times n$ 行列) の場合

1. **列方向に 0〜(n-1) 列目まで順に処理** する (行数が m の場合は $\min(m, n)$ 列まで)。
2. **各列のピボット(基準となる非0要素)を探し、行を上に持ってくる (必要なら入れ替え)**
3. **その行のピボットを 1 に規格化(行の定数倍)** して、**他の行の同じ列を 0 にする(行どうしの加減)**
4. 次の列に進む
5. 最終的に行列が「階段形」または「行簡約形」になれば、そこから元の連立方程式の解を求める(後退代入、または直接読取)

---

## 5. ピボット選択(pivoting)の重要性

実際の数値計算では、

* **「途中でピボットが 0 に近い数だと、丸め誤差が大きくなる」**
* **0 だと消去ができなくなる**
  といった問題があり、しばしば **部分的ピボット選択 (partial pivoting)** や **完全ピボット選択 (complete pivoting)** などを行います。
  これは「その列の下のほう(または行列全体)を調べて、いちばん大きな絶対値を持つ要素をピボットにする」→行や列を入れ替えて計算の安定性を確保する手法です。

---

## 6. ガウス=ジョルダン消去法

ガウスの消去法で得た「上三角形」から後退代入すれば解が求まりますが、**ガウス=ジョルダン消去法 (Gauss-Jordan)** ではさらに、**ピボット列の上の要素も 0 にして** **完全な対角行列(または単位行列)化** まで行います。
最終的に

$$
\begin{pmatrix}
I & \mathbf{x}
\end{pmatrix}
$$

のように変形して、解を直接得ることができる利点があります。
ただし計算量は増えるので、単に「連立方程式を1回だけ解きたい」という場合はガウス消去 + 後退代入で十分です。

---

## 7. ガウス消去法の計算量

サイズ $n \times n$ の連立方程式をガウス消去で解く場合、計算量(乗除算回数)は **おおむね $O(n^3)$** です。具体的には約 $\frac{2}{3}n^3$ オーダーの演算回数になります。この $O(n^3)$ という性質は、大規模行列で数値計算をするうえで基本指標になっています。

---

## 8. まとめ

1. **ガウスの消去法** は行基本変換を用いて、連立一次方程式を解く標準的な手法。
2. **手順**: (1) 上から順にピボットを決めて、下の要素を消す → 上三角形や階段形 → (2) 後退代入で解を求める。
3. **数値計算上** は丸め誤差の影響があるため、しばしば **ピボット選択** を行って安定性を高める。
4. **ガウス=ジョルダン消去法** では上下ともに消去して単位行列まで変換し、解を直接読み取ることもできる。

連立方程式を解く、行列のランクを調べる、逆行列を求める…など、線形代数の多くの応用でこのガウスの消去法が使われます。応用範囲の広い、重要なアルゴリズムです。


In [33]:
A3 = np.concatenate((A1, B1), axis=1)
print(np.linalg.matrix_rank(A3))
print(gaussian_elmination(A3))
A4=np.concatenate((A2,B2),axis=1)
print(gaussian_elmination(A4))

2
[0, 1]
[0, 1, 2]


In [52]:
#46,47クラメルの公式
def cramer(A,B):
    detA=np.linalg.det(A)
    tA=[r for r in zip(*A)]
    tB=[r for r in zip(*B)]
    res=[]
    for i in range(len(A)):
        p=tA[i]
        tA[i]=tB[0]
        ttA=[r for r in zip(*tA)]
        res.append(np.linalg.det(ttA)/detA)
        tA[i]=p
    return res

A=[[1,3],[4,2]]
B=[[2],[6]]
C=cramer(A,B)
print(C)
A2=[[1,2,3],[3,2,1],[2,3,1]]
B2=[[5],[6],[7]]
C2=cramer(A2,B2)
print(C2)

[np.float64(1.4000000000000001), np.float64(0.19999999999999996)]
[np.float64(0.7499999999999999), np.float64(1.75), np.float64(0.24999999999999986)]


In [56]:
#48,49 LU分解
Alu=scipy.linalg.lu(A2)
for mx in Alu:
    print(mx)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.8        1.        ]]
[[3.         2.         1.        ]
 [0.         1.66666667 0.33333333]
 [0.         0.         2.4       ]]


In [8]:
from scipy.linalg import lu_factor, lu_solve

# 係数行列 A
A = np.array([[2, 5, 8],
              [5, 2, 2],
              [7, 5, 6]])

# 定数項ベクトル b
b = np.array([23, 14, 32])

# 1. 行列 A を LU 分解する (lu_factor を使用)
# lu_factor は (LUを組み合わせた行列, ピボット情報) のタプルを返す
lu_and_piv = lu_factor(A)

# 2. LU 分解の結果と b を使って連立方程式を解く
x = lu_solve(lu_and_piv, b)
x2 = np.linalg.solve(A,b)
# 解を表示
print("連立方程式の解 x:\n", 'scipy:',x,'numpy:',x2)

# 検算: A @ x が b に等しくなるか確認
print("A @ x:\n", A @ x)
print("Is A @ x close to b?:", np.allclose(A @ x, b))

連立方程式の解 x:
 scipy: [-0.5  14.   -5.75] numpy: [-0.5  14.   -5.75]
A @ x:
 [23. 14. 32.]
Is A @ x close to b?: True


In [ ]:
#50連立方程式の数値誤差
#観察済み